In [ ]:
import asyncio
import ollama


In [5]:
client = ollama.AsyncClient()

In [ ]:
models = await client.list()

print("Available models:")
for model in models['models']:
    print(f"- {model}")

Available models:
- model='qwen3:1.7b' modified_at=datetime.datetime(2025, 5, 27, 11, 34, 2, 140572, tzinfo=TzInfo(+01:00)) digest='458ce03a21878025d072b8740d0e0e3b34abec3585fdadb0b5ab92d1e85dfb61' size=1359293203 details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='2.0B', quantization_level='Q4_K_M')
- model='codellama:latest' modified_at=datetime.datetime(2025, 5, 25, 14, 20, 53, 711605, tzinfo=TzInfo(+01:00)) digest='8fdf8f752f6e80de33e82f381aba784c025982752cd1ae9377add66449d2225f' size=3825910662 details=ModelDetails(parent_model='', format='gguf', family='llama', families=None, parameter_size='7B', quantization_level='Q4_0')
- model='phi3:mini' modified_at=datetime.datetime(2025, 5, 24, 19, 44, 11, 990643, tzinfo=TzInfo(+01:00)) digest='4f222292793889a9a40a020799cfd28d53f3e01af25d48e06c5e708610fc47e9' size=2176178913 details=ModelDetails(parent_model='', format='gguf', family='phi3', families=['phi3'], parameter_size='3.8B', quan

In [9]:
response = await client.generate('phi3:mini', 'Why is the sky blue?')
print(response['response'])

The sky appears blue to us due to a phenomenon called Rayleigh scattering. When sunlight reaches Earth'an atmosphere, it interacts with molecules and small particles in the air. Sunlight consists of different colors that correspond to various wavelengths of light. Shorter wavelengths (blue) are scattered more efficiently by these tiny atmospheric particulates than longer wavelengths like red or yellow because they travel as smaller, shorter waves; this scattering process spreads the blue light in many directions and when we look towards the sun, our line of sight passes through a greater volume of atmosphere where it has been scattered. Hence, with more direct path being taken by blues rays from different parts at higher angles around the horizon to us than other colors during daytime under clear sky conditions, this is why Earth's sky appears predominantly blue most of time while sun is up.


The effect diminishes towards twilight (before dawn) when there’s less contrast between direc

# Tooling 

In [14]:
response = await client.pull('qwen3:1.7b')
print(f"Model pull status: {response['status']}")

Model pull status: success


In [21]:
# Check model manifest and tagging for qwen3:1.7b
async def check_model_manifest():
    try:
        # Get model manifest
        manifest = await client.show('qwen3:1.7b')
        print("\nModel Manifest:")
        print(f"Model: {manifest}")
        print(f"Template: {manifest['template']}")
        print(f"Parameters: {manifest['parameters']}")
        print(f"Details: {manifest['details']}")
            
    except Exception as e:
        print(f"Error checking model manifest: {str(e)}")

# Run the check
await check_model_manifest()



Model Manifest:
Model: modified_at=datetime.datetime(2025, 5, 27, 11, 34, 2, 140572, tzinfo=TzInfo(+01:00)) template='{{- if .Messages }}\n{{- if or .System .Tools }}<|im_start|>system\n{{- if .System }}\n{{ .System }}\n{{- end }}\n{{- if .Tools }}\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{{- range .Tools }}\n{"type": "function", "function": {{ .Function }}}\n{{- end }}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call>\n{{- end }}<|im_end|>\n{{ end }}\n{{- range $i, $_ := .Messages }}\n{{- $last := eq (len (slice $.Messages $i)) 1 -}}\n{{- if eq .Role "user" }}<|im_start|>user\n{{ .Content }}<|im_end|>\n{{ else if eq .Role "assistant" }}<|im_start|>assistant\n{{ if .Content }}{{ .Content }}\n{{

In [23]:
from ollama import ChatResponse


def add_two_numbers(a: int, b: int) -> int:
  """
  Add two numbers

  Args:
    a (int): The first number
    b (int): The second number

  Returns:
    int: The sum of the two numbers
  """
  return a + b


def subtract_two_numbers(a: int, b: int) -> int:
  """
  Subtract two numbers
  """
  return a - b


# Tools can still be manually defined and passed into chat
subtract_two_numbers_tool = {
  'type': 'function',
  'function': {
    'name': 'subtract_two_numbers',
    'description': 'Subtract two numbers',
    'parameters': {
      'type': 'object',
      'required': ['a', 'b'],
      'properties': {
        'a': {'type': 'integer', 'description': 'The first number'},
        'b': {'type': 'integer', 'description': 'The second number'},
      },
    },
  },
}

messages = [{'role': 'user', 'content': 'What is three plus one?'}]
print('Prompt:', messages[0]['content'])

available_functions = {
  'add_two_numbers': add_two_numbers,
  'subtract_two_numbers': subtract_two_numbers,
}


client = ollama.AsyncClient()

response: ChatResponse = await client.chat(
  'qwen3:1.7b',
  messages=messages,
  tools=[add_two_numbers, subtract_two_numbers_tool],
)

if response.message.tool_calls:
  # There may be multiple tool calls in the response
  for tool in response.message.tool_calls:
    # Ensure the function is available, and then call it
    if function_to_call := available_functions.get(tool.function.name):
      print('Calling function:', tool.function.name)
      print('Arguments:', tool.function.arguments)
      output = function_to_call(**tool.function.arguments)
      print('Function output:', output)
    else:
      print('Function', tool.function.name, 'not found')

# Only needed to chat with the model using the tool call results
if response.message.tool_calls:
  # Add the function response to messages for the model to use
  messages.append(response.message)
  messages.append({'role': 'tool', 'content': str(output), 'name': tool.function.name})

  # Get final response from model with function outputs
  final_response = await client.chat('qwen3:1.7b', messages=messages)
  print('Final response:', final_response.message.content)

else:
  print('No tool calls returned from model')

Prompt: What is three plus one?
Calling function: add_two_numbers
Arguments: {'a': 3, 'b': 1}
Function output: 4
Final response: <think>
Okay, the user asked "What is three plus one?" So I need to calculate 3 + 1. Let me think. Three plus one is straightforward. The answer should be 4. I should check if there's any trick or special case, but since it's a simple addition, there's nothing to worry about. The previous response from the assistant was {"name": "add_two_numbers", "arguments": {"a":3,"b":1}}, which makes sense. The result was 4, so the final answer is 4. I need to present this in a clear and concise manner.
</think>

The result of three plus one is **4**.


Multi Model

In [26]:
response = await client.pull('llama3.2-vision:latest')
print(f"Model pull status: {response['status']}")

Model pull status: success


In [32]:
import base64
from ollama import chat

from pathlib import Path

# Pass in the path to the image
path = "/Users/admin/5nb_gifts/gifts/ZEN/working_docs/Product/master_data/hazops/Hazop.png"



In [33]:
# Pass in the path to the image
path = "/Users/admin/5nb_gifts/gifts/ZEN/working_docs/Product/master_data/hazops/Hazop.png"

# You can also pass in base64 encoded image data
img = base64.b64encode(Path(path).read_bytes()).decode()
# or the raw bytes
# img = Path(path).read_bytes()
?img

Type:        str
String form: iVBORw0KGgoAAAANSUhEUgAAC1YAAAeqCAYAAADRgAFfAAAMS2lDQ1BJQ0MgUHJvZmlsZQAASImVVwdYU8kWnltSIQQIREBK6 <...> BEAiBEAiBEAiBEAiBEAiBEAiBEAiBEAiBEAiBEAiBEAiBEAiBEAiBEAiBEAiBEAiBuwj8P8pVxLZfxAU7AAAAAElFTkSuQmCC
Length:      1516592
Docstring:  
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.

In [ ]:
import ollama
from pathlib import Path
import asyncio

# Path to your image
image_path = "/Users/admin/5nb_gifts/gifts/ZEN/working_docs/Product/master_data/hazops/Hazop.png"

# Read the image as bytes
img_bytes = Path(image_path).read_bytes()

# Use the async client for vision models
client = ollama.AsyncClient()

async def run_vision():
    try:
        response = await client.chat(
            model='llama3.2-vision:latest',
            messages=[
                {
                    'role': 'user',
                    'content': 'What is in this image? Be concise.',
                    'images': [img_bytes],  # Pass bytes, not path
                }
            ]
        )
        print(response.message.content)
    except Exception as e:
        print("Error:", e)

await run_vision()


In [ ]:
print(response.message.content)